In [ ]:
import pygame
pygame.display.quit()


from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../../")

import pygame
import pymunk

import numpy as np
import math

import gym
import cavs_environments
# import matplotlib

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines import PPO2
from stable_baselines import A2C
from stable_baselines import TRPO
from stable_baselines.common.policies import FeedForwardPolicy, register_policy
# from cavs_environments.vehicle.deep_road.deep_road import DeepRoad

import cavs_environments.framework as framework
import cavs_environments.vehicle.k_road.scenario.targeting as targeting
# import  cavs_environments.vehicle.k_road.targeting as

def make_target_env_with_baseline(
        observation_scaling=1.0,
        action_scaling=1.0 / 10.0,
        max_distance_from_target=125,
        time_limit=60):
    return framework.FactoredGym(
        targeting.TargetProcess(max_distance_from_target),
        targeting.TargetObserver(time_limit, observation_scaling),
        targeting.TargetTerminator(time_limit),
        targeting.TargetRewarder(),
        [framework.ActionScaler(action_scaling), targeting.TargetBaseline()]
    )


def make_target_env(
        observation_scaling=1.0,
        action_scaling=1.0 / 100.0,
        max_distance_from_target=125,
        time_limit=60):
    return framework.FactoredGym(
        targeting.TargetProcess(max_distance_from_target),
        targeting.TargetObserver(time_limit, observation_scaling),
        targeting.TargetTerminator(time_limit),
        targeting.TargetRewarder(),
        [framework.ActionScaler(action_scaling)]
    )

class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           net_arch=[16, 16, dict(pi=[16, 16, 16],
                                                          vf=[16, 16, 16])],
                                           feature_extraction="mlp")

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Linux (64bit) [/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/pymunk/libchipmunk.so]
Registered cavs_environments!


In [7]:
env = make_target_env_with_baseline()
env.reset()
env.render()

obs = None
for _ in range(10000):
    action = np.empty(2)
    action[0] = 0 # np.random.normal(.5, .001)
    action[1] = 0 # np.random.normal(.5, .001)
    
    result = env.step(action)
    obs = result[0]
    env.render()
    if result[2]:
        break
env.close()

KeyboardInterrupt: 

In [2]:
n_cpu = 12
env = SubprocVecEnv([lambda: make_target_env() for i in range(n_cpu)])
model = PPO2(CustomPolicy, env, verbose=1, tensorboard_log='/tmp/k_road_0/',
            gamma=.999, learning_rate=.0001)
model.learn(total_timesteps=int(5e6))
model.save('k_road_0')
print('done!')

W0823 17:06:33.412303 140134585775936 deprecation_wrapper.py:119] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/stable_baselines/common/tf_util.py:98: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0823 17:06:33.413890 140134585775936 deprecation_wrapper.py:119] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/stable_baselines/common/tf_util.py:107: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0823 17:06:33.488656 140134585775936 deprecation_wrapper.py:119] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/stable_baselines/common/policies.py:115: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0823 17:06:33.489406 140134585775936 deprecation_wrapper.py:119] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/stable_baselines/common/input.py:25: The na

---------------------------------------
| approxkl           | 7.0338e-06     |
| clipfrac           | 0.0            |
| explained_variance | -0.566         |
| fps                | 1952           |
| n_updates          | 1              |
| policy_entropy     | 2.8379636      |
| policy_loss        | -0.00015591482 |
| serial_timesteps   | 128            |
| time_elapsed       | 1.91e-06       |
| total_timesteps    | 1536           |
| value_loss         | 0.023252718    |
---------------------------------------
---------------------------------------
| approxkl           | 1.7642591e-05  |
| clipfrac           | 0.0            |
| explained_variance | -18            |
| fps                | 4199           |
| n_updates          | 2              |
| policy_entropy     | 2.8384318      |
| policy_loss        | -0.00044523377 |
| serial_timesteps   | 256            |
| time_elapsed       | 0.788          |
| total_timesteps    | 3072           |
| value_loss         | 6.19591e-05    |


---------------------------------------
| approxkl           | 4.0036728e-05  |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 3882           |
| n_updates          | 17             |
| policy_entropy     | 2.8481786      |
| policy_loss        | -0.00022048801 |
| serial_timesteps   | 2176           |
| time_elapsed       | 5.89           |
| total_timesteps    | 26112          |
| value_loss         | 9.0512134e-07  |
---------------------------------------
--------------------------------------
| approxkl           | 1.8004234e-06 |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 5044          |
| n_updates          | 18            |
| policy_entropy     | 2.8491907     |
| policy_loss        | -9.986963e-06 |
| serial_timesteps   | 2304          |
| time_elapsed       | 6.29          |
| total_timesteps    | 27648         |
| value_loss         | 8.964848e-07  |
------------

---------------------------------------
| approxkl           | 1.3359337e-05  |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 4662           |
| n_updates          | 33             |
| policy_entropy     | 2.8565753      |
| policy_loss        | -0.00022579072 |
| serial_timesteps   | 4224           |
| time_elapsed       | 11.1           |
| total_timesteps    | 50688          |
| value_loss         | 2.0286757e-07  |
---------------------------------------
---------------------------------------
| approxkl           | 1.859781e-05   |
| clipfrac           | 0.0            |
| explained_variance | 5.96e-08       |
| fps                | 4178           |
| n_updates          | 34             |
| policy_entropy     | 2.8559687      |
| policy_loss        | -0.00010980072 |
| serial_timesteps   | 4352           |
| time_elapsed       | 11.4           |
| total_timesteps    | 52224          |
| value_loss         | 1.1310872e-07  |


--------------------------------------
| approxkl           | 4.493563e-06  |
| clipfrac           | 0.0           |
| explained_variance | -6.79e-06     |
| fps                | 4925          |
| n_updates          | 49            |
| policy_entropy     | 2.862964      |
| policy_loss        | -4.053562e-06 |
| serial_timesteps   | 6272          |
| time_elapsed       | 16.4          |
| total_timesteps    | 75264         |
| value_loss         | 7.0441223e-07 |
--------------------------------------
--------------------------------------
| approxkl           | 1.728651e-06  |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 4300          |
| n_updates          | 50            |
| policy_entropy     | 2.8638062     |
| policy_loss        | -3.333281e-05 |
| serial_timesteps   | 6400          |
| time_elapsed       | 16.7          |
| total_timesteps    | 76800         |
| value_loss         | 3.1784634e-08 |
-------------------------

...

---------------------------------------
| approxkl           | 1.6341937e-06  |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 5279           |
| n_updates          | 3178           |
| policy_entropy     | 4.1609826      |
| policy_loss        | -5.3661304e-05 |
| serial_timesteps   | 406784         |
| time_elapsed       | 974            |
| total_timesteps    | 4881408        |
| value_loss         | 8.757256e-08   |
---------------------------------------
---------------------------------------
| approxkl           | 6.998866e-06   |
| clipfrac           | 0.0            |
| explained_variance | -0.642         |
| fps                | 5062           |
| n_updates          | 3179           |
| policy_entropy     | 4.160879       |
| policy_loss        | -0.00021902595 |
| serial_timesteps   | 406912         |
| time_elapsed       | 974            |
| total_timesteps    | 4882944        |
| value_loss         | 6.213909e-05   |


--------------------------------------
| approxkl           | 1.7519073e-06 |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 5259          |
| n_updates          | 3194          |
| policy_entropy     | 4.1634707     |
| policy_loss        | -3.210208e-05 |
| serial_timesteps   | 408832        |
| time_elapsed       | 979           |
| total_timesteps    | 4905984       |
| value_loss         | 1.0105687e-07 |
--------------------------------------
--------------------------------------
| approxkl           | 1.1941884e-06 |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 5017          |
| n_updates          | 3195          |
| policy_entropy     | 4.1651244     |
| policy_loss        | -3.481282e-05 |
| serial_timesteps   | 408960        |
| time_elapsed       | 979           |
| total_timesteps    | 4907520       |
| value_loss         | 9.0782265e-08 |
-------------------------

---------------------------------------
| approxkl           | 1.4360174e-06  |
| clipfrac           | 0.0            |
| explained_variance | 1.79e-07       |
| fps                | 5278           |
| n_updates          | 3210           |
| policy_entropy     | 4.171493       |
| policy_loss        | -4.2364685e-05 |
| serial_timesteps   | 410880         |
| time_elapsed       | 984            |
| total_timesteps    | 4930560        |
| value_loss         | 1.08331385e-07 |
---------------------------------------
---------------------------------------
| approxkl           | 1.5558919e-05  |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 5104           |
| n_updates          | 3211           |
| policy_entropy     | 4.172575       |
| policy_loss        | -0.000277394   |
| serial_timesteps   | 411008         |
| time_elapsed       | 984            |
| total_timesteps    | 4932096        |
| value_loss         | 1.12409815e-07 |


---------------------------------------
| approxkl           | 2.6291743e-06  |
| clipfrac           | 0.0            |
| explained_variance | -2.07          |
| fps                | 5074           |
| n_updates          | 3226           |
| policy_entropy     | 4.1824927      |
| policy_loss        | -0.00012648609 |
| serial_timesteps   | 412928         |
| time_elapsed       | 989            |
| total_timesteps    | 4955136        |
| value_loss         | 2.1714597e-05  |
---------------------------------------
--------------------------------------
| approxkl           | 1.291592e-06  |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 4996          |
| n_updates          | 3227          |
| policy_entropy     | 4.182223      |
| policy_loss        | -7.418353e-05 |
| serial_timesteps   | 413056        |
| time_elapsed       | 989           |
| total_timesteps    | 4956672       |
| value_loss         | 1.3914124e-07 |
------------

---------------------------------------
| approxkl           | 1.0242964e-05  |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 5014           |
| n_updates          | 3242           |
| policy_entropy     | 4.1770015      |
| policy_loss        | -5.7480902e-05 |
| serial_timesteps   | 414976         |
| time_elapsed       | 994            |
| total_timesteps    | 4979712        |
| value_loss         | 9.967784e-08   |
---------------------------------------
---------------------------------------
| approxkl           | 8.534298e-06   |
| clipfrac           | 0.0            |
| explained_variance | -1.19e-07      |
| fps                | 5215           |
| n_updates          | 3243           |
| policy_entropy     | 4.17686        |
| policy_loss        | -0.00014676935 |
| serial_timesteps   | 415104         |
| time_elapsed       | 994            |
| total_timesteps    | 4981248        |
| value_loss         | 9.529279e-08   |


In [12]:
env = make_target_env()
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run
model = PPO2.load('k_road_0')
obs = env.reset()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, terminal, info = env.step(action)
    env.render()
    if terminal:
        obs = env.reset()
        

Loading a model without an environment, this model cannot be trained until it has a valid environment.


KeyboardInterrupt: 

In [ ]:
n_cpu = 10
env = SubprocVecEnv([lambda: make_target_env() for i in range(n_cpu)])
model = A2C(MlpPolicy, env, verbose=1, tensorboard_log='/tmp/k_road_tensorboard_0/')
model.learn(total_timesteps=int(150e3))
model.save('k_road_a2c_1')
print('done!')

In [ ]:
env = make_target_env()
env = DummyVecEnv([lambda: env])
model = A2C.load('k_road_a2c_3')
obs = env.reset()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, terminal, info = env.step(action)
    env.render()
    if terminal:
        break

In [ ]:
env = make_target_env()
env = DummyVecEnv([lambda: env])
model = TRPO(MlpPolicy, env, verbose=1, tensorboard_log='/tmp/k_road_tensorboard_0/',
            gamma=.999,
            timesteps_per_batch = 5 * 60 * 10)
model.learn(total_timesteps=int(70e3))
model.save('k_road_1')
print('done!')

In [ ]:
env = make_target_env()
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run
model = TRPO.load('k_road_1')
obs = env.reset()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, terminal, info = env.step(action)
    env.render()
    if terminal:
        break

In [ ]:
def two_sided_exponential(gain, x):
    return math.copysign(1, x) * (math.exp(gain * (math.fabs(x) - 1)) - math.exp(-gain)) / (1 - math.exp(-gain))

def inverse_two_sided_exponential(gain, y):
    z = math.fabs(y)
    return math.copysign(1, y) * math.log(z * math.exp(gain) - z + 1) / gain

def two_sided_offset_exponential(gain, offset, x):
    y = 0
    s = 1
    if x < offset:
        y = 1 - (x + 1) / (offset + 1)
        s = -1
    else:
        y = (x - offset) / (1 - offset)
    return  s*(math.exp(gain * y)  - 1) / (math.exp(gain) - 1)

def linear_offset(offset, x):
    if x < offset:
        return (x + 1) / (offset + 1) - 1
    else:
        return (x - offset) / (1 - offset)
        
def linear_offset_exponential(gain, offset, x):
    if offset > 0:
        if x < offset:
            return linear_offset(offset, x)
        else:
            return two_sided_offset_exponential(gain, offset, x)
    else:
        if x < offset:
            return two_sided_offset_exponential(gain, offset, x)
        else:
            return linear_offset(offset, x)
        
def exponential_transform(gain, x):
    y = (x + 1) / 2 
    return 2 * (math.exp(gain * y) - 1) / (math.exp(gain) - 1) - 1
        
k = 2
x = np.arange(-1.0,1.0,.001)
# x = np.arange(0, 1, .01)
# matplotlib.pyplot.plot(x, np.array([two_sided_offset_exponential(k, .5, xi) for xi in x]))

# matplotlib.pyplot.plot(x, np.array([two_sided_exponential(k, xi) for xi in x]))

# matplotlib.pyplot.plot(x, np.array([linear_offset(.5, xi) for xi in x]))

# matplotlib.pyplot.plot(x, np.array([linear_offset_exponential(2, -.5, xi) for xi in x]))

matplotlib.pyplot.plot(x, np.array([exponential_transform(2, xi) for xi in x]))

print(exponential_transform(2, -1))

# y = np.array([inverse_two_sided_exponential(k, xi) for xi in x])
# matplotlib.pyplot.plot(x, y)
